# 🔬 Hands‑On Persian NLP with 🤗 Transformers

This notebook walks you through **three core tasks**—Named‑Entity Recognition, Sentiment
Analysis, and Text Generation—using publicly‑available Persian checkpoints on
Hugging Face.  
All code cells are complete and runnable; feel free to tweak the examples or
replace the input strings with your own Persian text.


## 🛠 0  Environment Setup


In [ ]:
# Core Hugging Face stack
!pip install --upgrade transformers accelerate sentencepiece -q


## 📑 1  Named‑Entity Recognition (NER) with BERT‑fa
We will load a BERT‑based sequence‑labeling model fine‑tuned on
the PEYMA news corpus.

The pipeline automatically aggregates sub‑token predictions into whole
entities.

In [ ]:
#Research task 1
# 1. Go to the Hugging Face Model Hub and search for a Persian NER checkpoint (hint: keywords “bert‑fa” and “ner”).
# 2. Read the model card to note its model‑id.
# 3. In a new cell, create a pipeline("ner", model=MODEL_ID, aggregation_strategy="simple").
# 4. Route it to GPU if available (device=0) otherwise stay on CPU.
# 5. Test the pipeline on a Persian sentence of your choice and pretty‑print the entities.

!pip install transformers torch
from transformers import pipeline
import pprint
import torch

MODEL_ID = "HooshvareLab/bert-fa-base-uncased-ner-peyma"
device = 0 if torch.cuda.is_available() else -1

ner = pipeline(
    task="ner",
    model=MODEL_ID,
    aggregation_strategy="simple",
    device=device
)


In [2]:
test_sentence = "دکتر سارا گل‌محمدی در تهران سخنرانی کرد."
entities = ner(test_sentence)

print("Input:", test_sentence)
print("Detected entities:")
pprint.pp(entities, compact=True)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Input: دکتر سارا گل‌محمدی در تهران سخنرانی کرد.
Detected entities:
[{'entity_group': 'B_PER',
  'score': np.float32(0.999181),
  'word': 'سارا',
  'start': 5,
  'end': 9},
 {'entity_group': 'I_PER',
  'score': np.float32(0.9997402),
  'word': 'گلمحمدی',
  'start': 10,
  'end': 18},
 {'entity_group': 'B_LOC',
  'score': np.float32(0.99893755),
  'word': 'تهران',
  'start': 22,
  'end': 27}]


## 😊 2  Sentiment Analysis with BERT‑fa (Digikala Reviews)
This checkpoint predicts POSITIVE or NEGATIVE sentiment on short
product reviews.

We will score a handful of sample sentences.



In [ ]:
# Research task 2
# 1. Browse the Hugging Face Hub for a Persian sentiment‑analysis model (hint: search “bert‑fa sentiment digikala” or “parsbert sentiment”).
# 2. Copy its model‑id from the model card.
# 3. Build a pipeline("sentiment-analysis", model=MODEL_ID, device=0|‑1) in a new cell.
# 4. Run the pipeline on at least four Persian reviews (two positive, two negative) and print the predicted labels.
# 5. Briefly comment on whether the predictions match your intuition.

In [3]:
from transformers import pipeline

MODEL_ID = "HooshvareLab/bert-fa-base-uncased-sentiment-digikala"
sentiment_pipe = pipeline("sentiment-analysis", model=MODEL_ID)

reviews = [
    "این گوشی واقعاً بی‌نظیره!",  # "This phone is truly exceptional!"
    "طراحی زیبا ولی باتری خیلی ضعیفه.",  # "Beautiful design but very weak battery."
    "کیفیت ساختش عالیه و ارزش خرید داره.",  # "Its build quality is excellent and worth buying."
    "قیمت نسبت به امکاناتش خیلی بالاست.",  # "The price is too high for its features."
]

for r in reviews:
    result = sentiment_pipe(r)[0]
    print(f"{r} → {result['label']} (confidence: {result['score']:.2f})")



config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/651M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/651M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


این گوشی واقعاً بی‌نظیره! → recommended (confidence: 0.98)
طراحی زیبا ولی باتری خیلی ضعیفه. → no_idea (confidence: 0.74)
کیفیت ساختش عالیه و ارزش خرید داره. → recommended (confidence: 0.97)
قیمت نسبت به امکاناتش خیلی بالاست. → no_idea (confidence: 0.58)


## ✍️ 3  Text Generation with GPT‑2‑fa
GPT2‑fa is a 124 M‑parameter GPT‑2 adapted to 500 M Persian tokens.
Below we build a system + user prompt, generate up to 200 new tokens, and print the response after stripping the prompt.

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "HooshvareLab/gpt2-fa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
gpt2fa = AutoModelForCausalLM.from_pretrained(model_name).to(
    "cuda" if torch.cuda.is_available() else "cpu"
)

system_instruction = "شما یک دستیار فارسی هستید که به سؤالات با لحن دوستانه پاسخ می‌دهید."
user_prompt = "سلام! میشه دربارهٔ هوش مصنوعی توضیح بدی؟"
full_prompt = f"{system_instruction}\n\n{user_prompt}"

inputs = tokenizer(full_prompt, return_tensors="pt").to(gpt2fa.device)
outputs = gpt2fa.generate(
    **inputs,
    max_new_tokens=200,
    top_p=0.9,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
assistant_reply = decoded[len(full_prompt):].strip()

print("🟢 Assistant reply:\n")
print(assistant_reply)


🟢 Assistant reply:

ما از هوش مصنوعی استفاده می‌کنیم. به این معنی که ما به جای اینکه دربارهٔ یک موضوع خاص صحبت کنیم، دربارهٔ آن موضوع صحبت می‌کنیم. مثلا به جای اینکه راجع به موضوعی خاص صحبت کنیم، دربارهٔ موضوع مورد بحث صحبت می‌کنیم. به این صورت که اگر در مورد موضوعی صحبت می‌کنیم، دربارهٔ آن موضوع صحبت می‌کنیم. به این معنی که به جای اینکه دربارهٔ موضوعی صحبت کنیم، دربارهٔ آن موضوع صحبت می‌کنیم. به این معنی که راجع به موضوعی صحبت می‌کنیم که دربارهٔ آن صحبت می‌کنیم. به این معنی که وقتی راجع به موضوعی صحبت می‌کنیم، راجع به آن موضوع صحبت می‌کنیم. به این معنی که وقتی راجع به موضوعی صحبت می‌کنیم، راجع به آن موضوع صحبت می‌کنیم. به این معنی که وقتی راجع به موضوعی صحبت می‌کنیم، راجع به آن موضوع صحبت می‌کنیم. به این معنی که وقتی راجع به موضوعی صحبت می‌کنیم، راجع به


## 🎯 4  End‑to‑End Demo: Generate → Sentiment → NER
The function below:

* Generates text from a short seed using GPT‑2‑fa.

* Predicts the overall sentiment of the generated paragraph.

* Extracts named entities inside that text.


In [6]:
def generate_analyse(seed: str, max_new: int = 120):
    full_prompt = system_instruction + "\n\n" + seed
    input_ids   = tokenizer(full_prompt, return_tensors="pt").to(
        gpt2fa.device
    )
    gen_ids = gpt2fa.generate(
        **input_ids,
        max_new_tokens=max_new,
        do_sample=True,
        top_p=0.9,
        temperature=0.85,
    )
    decoded   = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
    generated = decoded[len(full_prompt):].strip()

    sentiment = sentiment_pipe(generated)[0]
    entities  = ner(generated)

    return {
        "generated_text": generated,
        "sentiment": sentiment,
        "entities": entities,
    }

result = generate_analyse("یک مشتری در مورد لپ‌تاپ جدید ایسوس اینگونه نوشت:")
print("📜 Generated text:\n", result["generated_text"])
print("\n🔶 Sentiment:", result["sentiment"])
print("\n🔷 Entities:")
pprint.pp(result["entities"], compact=True)


Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.


📜 Generated text:
 «ما در این لپ‌تاپ از پردازنده‌های نسل ششم اینتل و کارت گرافیک انویدیا استفاده کرده‌ایم و تمام تلاش خود را برای استفاده حداکثری از توان گرافیکی آن به کار برده‌ایم تا تجربه کار با لپ‌تاپ‌های نسل ششم را بیش از پیش بهبود بخشیم.» ایسوس برای اولین بار در تاریخ خود، یک لپ‌تاپ مجهز به پردازنده‌های نسل ششم اینتل را با عنوان UX305 معرفی کرده است. در نگاه اول، UX305 در مقایسه با UX305 دارای قیمت پایین‌تری است و در مقایسه با UX305 قیمت بالاتری دارد. نمایشگر UX

🔶 Sentiment: {'label': 'recommended', 'score': 0.7739930748939514}

🔷 Entities:
[{'entity_group': 'B_ORG',
  'score': np.float32(0.7389317),
  'word': 'اینتل',
  'start': 42,
  'end': 47},
 {'entity_group': 'B_ORG',
  'score': np.float32(0.7009733),
  'word': 'انویدیا',
  'start': 62,
  'end': 69},
 {'entity_group': 'B_ORG',
  'score': np.float32(0.9968586),
  'word': 'ایسوس',
  'start': 225,
  'end': 230},
 {'entity_group': 'B_ORG',
  'score': np.float32(0.82838356),
  'word': 'اینتل',
  'start': 299,
  'end': 304}]


## 📝 5  Reflection (write your answers in Markdown)
1. Which task—NER, sentiment, or generation—appeared most robust on your custom inputs? Why?

2. How could you improve sentiment accuracy on longer, mixed‑tone reviews?

3. Suggest a real‑world product idea that chains generation + sentiment + NER.